# Lab 5: Leveraging Open Data from Wikipedia for LLM Prompt Engineering

## Overview
This lab demonstrates how to extract structured data from Wikipedia pages and use it to create effective prompts for Large Language Models (LLMs). You'll learn to work with real-world financial data, process it programmatically, and engineer prompts for various AI tasks.

## Learning Objectives
- ✓ Extract financial index components from Wikipedia
- ✓ Retrieve company infobox data programmatically
- ✓ Build structured datasets from semi-structured web data
- ✓ Design effective LLM prompts for different tasks
- ✓ Process and clean text data for AI consumption
- ✓ Create reusable prompt templates and utilities

## Part 1: Data Extraction from Wikipedia

### What is a Financial Index?
A financial index is a composite measure of a subset of companies in a specific market or sector. Examples include:
- **S&P 500**: 500 largest US companies
- **EURO STOXX 50**: 50 largest Eurozone companies
- **DAX**: 40 largest German companies

### Your Task
1. **Identify components**: Extract the list of companies in each index from Wikipedia
2. **Gather company data**: Retrieve detailed information (infoboxes) from each company's Wikipedia page
3. **Build a dataset**: Combine all data into structured format suitable for LLM processing
4. **Engineer prompts**: Create effective prompts that leverage this data for AI tasks

### Data Sources
- **Index components**: Wikipedia articles listing index members
- **Company data**: Wikipedia infoboxes (structured data boxes on company pages)
- **Dump file**: Optional - for advanced analysis of full Wikipedia articles

### Optional: Full Wikipedia Dump
For advanced analysis, you can download the complete Wikipedia dump from:
- **Link**: https://dumps.wikimedia.org/enwiki/
- **File**: `enwiki-latest-pages-articles-multistream-index.txt.bz2`
- **Use case**: Full-text search, article history analysis, or complete data scraping
- **Note**: Very large files (100+ GB) - requires significant storage and processing power

For this lab, we'll focus on extracting specific data via the Wikipedia API, which is more efficient.

In [47]:
# ============================================================================
# IMPORTS & SETUP
# ============================================================================
# These libraries enable us to work with Wikipedia data

import pandas as pd              # Data manipulation and analysis
import urllib.request           # HTTP requests to Wikipedia
from pathlib import Path        # Cross-platform file path handling
from typing import Union        # Type hints for better code clarity
from tqdm import tqdm          # Progress bars for long operations
import wptools               # Wikipedia parsing (infobox extraction)
from loguru import logger       # Enhanced logging
import json                     # Working with JSON data
import numpy as np             # Numerical operations

## Step 1: Extract Index Components from Wikipedia

### Task: Extract Company Lists
We'll extract the list of companies that make up each financial index directly from Wikipedia.

### Indices We're Covering:
1. **S&P 500** (USA) - 500 largest US companies
2. **EURO STOXX 50** (Europe) - 50 largest Eurozone companies  
3. **CAC 40** (France) - 40 largest French companies
4. **DAX** (Germany) - 40 largest German companies
5. **CSI 300** (China) - 300 largest Chinese companies
6. **S&P Latin America 40** (Latin America) - 40 major LA companies
7. **BSE SENSEX** (India) - 30 largest Indian companies
8. **NASDAQ-100** (USA Tech) - 100 largest non-financial NASDAQ companies

### How It Works:
- Each index has a Wikipedia article with a table listing its components
- We'll use `pd.read_html()` to extract all tables from these pages
- Tables are saved as CSV files for later processing
- This approach is fast, requires no authentication, and respects Wikipedia's terms

In [48]:
# ============================================================================
# 1. GET TABLES
# ============================================================================

def download_tables(url, save_dir):
    """Downloads all tables from a Wiki page to CSVs."""
    path = Path(save_dir)
    path.mkdir(parents=True, exist_ok=True)
    
    print(f"Reading {url}...")
    # Pandas does the heavy lifting
    tables = pd.read_html(url)
    
    for i, df in enumerate(tables):
        df.to_csv(path / f"table_{i}.csv", index=False)
    print(f"-> Saved {len(tables)} tables in {save_dir}")


In [49]:
# ============================================================================
# SETUP: Configure Wikipedia Index URLs and HTTP Headers
# ============================================================================

# Dictionary mapping index names to their Wikipedia article URLs
# These URLs contain tables with the company components of each index
indices = {
    "sp500": "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies",
    "eurostoxx50": "https://en.wikipedia.org/wiki/EURO_STOXX_50",
    "cac40": "https://en.wikipedia.org/wiki/CAC_40",
    "dax": "https://en.wikipedia.org/wiki/DAX",
    "csi300": "https://en.wikipedia.org/wiki/CSI_300_Index",
    "spla40": "https://en.wikipedia.org/wiki/S%26P_Latin_America_40",
    "bsesensex": "https://en.wikipedia.org/wiki/BSE_SENSEX",
    "nasdaq100": "https://en.wikipedia.org/wiki/Nasdaq-100",
}

# IMPORTANT: Configure HTTP headers to identify our bot to Wikipedia
# This is REQUIRED for ethical web scraping - identify yourself!
# Wikipedia may block requests without proper User-Agent headers

opener = urllib.request.build_opener()
opener.addheaders = [
    ("User-Agent", "MyResearchBot/1.0 (contact@example.com)")  # Identify your bot
]
urllib.request.install_opener(opener)

In [50]:
# ============================================================================
# EXECUTE STEP 1
# ============================================================================

for name, url in indices.items():
    download_tables(url, f"data/indices/{name}")


Reading https://en.wikipedia.org/wiki/List_of_S%26P_500_companies...
-> Saved 3 tables in data/indices/sp500
Reading https://en.wikipedia.org/wiki/EURO_STOXX_50...
-> Saved 10 tables in data/indices/eurostoxx50
Reading https://en.wikipedia.org/wiki/CAC_40...
-> Saved 20 tables in data/indices/cac40
Reading https://en.wikipedia.org/wiki/DAX...
-> Saved 10 tables in data/indices/dax
Reading https://en.wikipedia.org/wiki/CSI_300_Index...
-> Saved 7 tables in data/indices/csi300
Reading https://en.wikipedia.org/wiki/S%26P_Latin_America_40...
-> Saved 4 tables in data/indices/spla40
Reading https://en.wikipedia.org/wiki/BSE_SENSEX...
-> Saved 33 tables in data/indices/bsesensex
Reading https://en.wikipedia.org/wiki/Nasdaq-100...
-> Saved 19 tables in data/indices/nasdaq100


## Step 2: Extract Company Infoboxes from Wikipedia

### What are Infoboxes?
Wikipedia infoboxes are structured data boxes that appear on the right side of articles. They contain:
- Company name and alternative names
- Industry classification
- Founded date and location
- Key executives
- Headquarters location
- Number of employees
- Revenue and financial metrics
- Official website URLs
- Stock exchange listings
- And much more...

### Why Infoboxes?
- **Structured data**: Unlike article body text, infoboxes are semi-structured
- **Consistency**: Fields follow a template across similar articles
- **Ease of extraction**: Wikipedia APIs can parse infoboxes directly
- **Rich context**: Perfect for LLM prompts - contains exactly the info LLMs need

### Process
1. Use the `wptools` library to fetch each company's Wikipedia page
2. Extract the infobox (structured data) from the page parse
3. Save as JSON for flexibility and later processing
4. Handle errors gracefully (some companies may not have Wikipedia pages)

In [51]:
# ============================================================================
# 2. GET INFOBOXES
# ============================================================================

def download_infobox(company, save_dir):
    """Gets the infobox for a company."""
    try:
        # Handle names like 'Alphabet (Class A)' -> 'Alphabet'
        search_name = company.split('(')[0].strip()
        
        # Fetch page
        page = wptools.page(search_name, silent=True).get_parse()
        infobox = page.data.get('infobox')
        
        if infobox:
            # Save to JSON
            clean_name = "".join(x for x in company if x.isalnum()).strip()
            path = Path(save_dir) / f"{clean_name}.json"
            path.parent.mkdir(parents=True, exist_ok=True)
            
            with open(path, 'w') as f:
                json.dump(infobox, f, indent=4)
            print(f"✓ {company}")
        else:
            print(f"✗ {company} (no infobox)")
            
    except Exception:
        print(f"✗ {company} (not found)")


In [52]:
# ============================================================================
# DISPLAY: View the Extracted Infobox
# ============================================================================
# This shows what data we extracted from Wikipedia

example_file = Path("data/infoboxes/sp500/3M.json")

if example_file.exists():
    with open(example_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        # Display the first few keys
        print("Infobox keys found:", list(data.keys())[:10])
        print("\nFull Content (truncated):\n")
        print(json.dumps(data, indent=4)[:500] + "\n...[truncated]")
else:
    print("File not found! Run the cell above first.")


File not found! Run the cell above first.


In [53]:
# ============================================================================
# EXECUTION: Batch Extract Infoboxes
# ============================================================================
# We'll extract a few companies from each index to test our pipeline.
# To extract ALL companies, set LIMIT = None (takes much longer!)

LIMIT = 10  # We'll only take the first 10 companies per index for this lab

# Configuration: which file and column to use for each index
index_config = {
    "sp500":       ("table_1.csv", "Security"),
    "eurostoxx50": ("table_4.csv", "Name"),
    "cac40":       ("table_4.csv", "Company"),
    "dax":         ("table_4.csv", "Company"),
    "csi300":      ("table_3.csv", "Company"),
    "nasdaq100":   ("table_4.csv", "Company")
}

for index_name, (csv_file, col_name) in index_config.items():
    print(f"\n=== Processing {index_name} ===")
    
    # 1. Load the company list
    csv_path = Path(f"data/indices/{index_name}/{csv_file}")
    if not csv_path.exists():
        print(f"CSV not found: {csv_path}")
        continue
        
    df = pd.read_csv(csv_path)
    companies = df[col_name].tolist()
    
    # 2. Extract infoboxes (limited number)
    target_dir = f"data/infoboxes/{index_name}"
    
    # Apply limit if set
    companies_to_process = companies[:LIMIT] if LIMIT else companies
    
    print(f"Extracting {len(companies_to_process)} companies...")
    
    for company in companies_to_process:
        get_company_infobox(company, target_dir)

print("\nBatch extraction complete!")



=== Processing sp500 ===
Extracting 10 companies...
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.

=== Processing eurostoxx50 ===
Extracting 10 companies...
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.

=== Processing cac40 ===
Extracting 10 companies...
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.

=== Processing dax ===
Extracting 10 companies...
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.
-> No infobox found for Continental

=== Pr

API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}


   -> Retrying with simplified name: 'Alphabet Inc.'...
-> Success! Saved.


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}


   -> Retrying with simplified name: 'Alphabet Inc.'...
-> Success! Saved.
-> No infobox found for Amazon
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.
-> Success! Saved.

Batch extraction complete!


## Step 3: Aggregate Infoboxes into Databases

### What We're Building
We're converting individual JSON files (one per company) into consolidated CSV databases (one per index).

### Why?
- **Easier analysis**: CSV format works with pandas, Excel, and most analysis tools
- **Efficiency**: One file per index instead of hundreds of individual JSON files
- **Standardization**: Creates a uniform dataset structure for LLM processing

### Process
1. Read all JSON infobox files for an index from disk
2. Convert each JSON to a DataFrame row
3. Concatenate all rows into a single DataFrame
4. Save as CSV with proper encoding

### Notes for Future Enhancement
- The infoboxes contain many fields beyond what we use now (URLs, images, etc.)
- Future work could extract and leverage additional information
- This foundation allows flexible data extraction later

In [54]:
# ============================================================================
# 3. MERGE TO CSV
# ============================================================================

def merge_to_csv(index_name):
    """Merges all JSONs in a folder to one CSV."""
    data = []
    folder = Path(f"data/infoboxes/{index_name}")
    
    # Read all JSONs
    for file in folder.glob("*.json"):
        with open(file) as f:
            row = json.load(f)
            row['file_id'] = file.stem # Keep track of source
            data.append(row)
            
    if data:
        # Save
        df = pd.DataFrame(data)
        out = Path(f"data/processed/{index_name}_data.csv")
        out.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(out, index=False)
        print(f"Created {out} ({len(df)} rows)")

# Run
for index in ["sp500", "cac40"]:
    merge_to_csv(index)


Created data/processed/sp500_data.csv (10 rows)
Created data/processed/cac40_data.csv (10 rows)


# Part 2: Data Processing & LLM Prompt Engineering

## Overview of Part 2
Now that we have structured company data from Wikipedia, we'll:

1. **Load and analyze** the infobox database
2. **Clean and preprocess** the data for LLM consumption
3. **Create prompt templates** for different LLM tasks
4. **Design context formatting** that maximizes LLM effectiveness

## Key Concepts

### Why Clean Data for LLMs?
- LLMs perform better with well-structured, clean text
- Removing noise and formatting artifacts improves accuracy
- Consistent formatting allows for better prompt engineering
- Clean data enables batch processing and cost optimization

### Prompt Engineering
Prompt engineering is the art of crafting inputs to LLMs to get better outputs. We'll explore:
- **Context formatting**: How to present company data effectively
- **Task-specific templates**: Different prompts for different goals
- **Few-shot learning**: Providing examples to guide LLM behavior
- **Output structuring**: Getting structured responses (JSON, tables, etc.)


In [55]:
# Load data
df = pd.read_csv("data/processed/sp500_data.csv")
print("Columns:", list(df.columns[:5]))
display(df.head(1))


Columns: ['name', 'logo', 'logo_size', 'image', 'image_size']


,name,logo,logo_size,image,image_size,image_caption,former_name,type,traded_as,ISIN,...,net_income_year,assets_year,equity_year,hq_location,footnotes,logo_caption,subsid,predecessor,num_locations,locations
0,3M Company,3M wordmark.svg,175px,3-M Building Maplewood MN1.jpg,250px,"3M headquarters in [[Maplewood, Minnesota]]",Minnesota Mining and Manufacturing Company (19...,[[Public company|Public]],{{Unbulleted list|New York Stock Exchange|MMM|...,{{ISIN|sl|=|n|pl|=|y|US88579Y1010}},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
# ============================================================================
# 4. PREPARE FOR AI
# ============================================================================

def clean(text):
    """Remove {{brackets}} and [[links]]."""
    if not isinstance(text, str): return "Unknown"
    for char in "[]{}":
        text = text.replace(char, "")
    return text.strip()

def make_context(row):
    """Create a readable profile from a row."""
    # Smart lookup: try different common column names
    name = row.get('name') or row.get('company_name') or row.get('file_id')
    industry = row.get('industry') or row.get('sector')
    hq = row.get('location') or row.get('headquarters')
    
    return f"""COMPANY PROFILE:
Name: {clean(name)}
Industry: {clean(industry)}
Location: {clean(hq)}
"""

# Test
print(make_context(df.iloc[0]))


COMPANY PROFILE:
Name: 3M Company
Industry: Conglomerate (company)|Conglomerate
Location: Unknown



In [57]:
# ============================================================================
# 5. GENERATE PROMPTS
# ============================================================================

SUMMARY_TASK = "Summarize this company in 1 sentence."
RISK_TASK = "Classify the risk level (Low/Med/High) based on this profile."

def create_prompt(row, task):
    """Combines Context + Task."""
    context = make_context(row)
    return f"{task}\n\nDATA:\n{context}"

# Demo
row = df.iloc[0]
print("--- PROMPT FOR AI ---")
print(create_prompt(row, SUMMARY_TASK))


--- PROMPT FOR AI ---
Summarize this company in 1 sentence.

DATA:
COMPANY PROFILE:
Name: 3M Company
Industry: Conglomerate (company)|Conglomerate
Location: Unknown



End of lab 5

🚀 NEXT STEPS (in anticipation of the final lab)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. Export your full dataset using PromptExporter
2. Test prompts with a small sample (5-10 companies)
3. Evaluate LLM outputs for quality and accuracy
4. Iterate on prompts based on results
5. Scale up to full dataset using batch APIs
6. Monitor token usage and costs
7. Implement feedback loops for continuous improvement
8. Build evaluation metrics for output quality